### Creating and analyzing time-distance plots with stackplotX.py

In [1]:
%matplotlib inline
%matplotlib notebook
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import os, sys, re
import pickle

import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta

import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
import warnings
warnings.filterwarnings("ignore")
sys.path.insert(0, '/home/ivan/Study/Astro/solar')
import matplotlib.colors as colors

In [3]:
'''
To get stackplotX working properly in sunpy 5.0
replace the line
from sunpy.physics.solar_rotation import mapsequence_solar_derotate
with
from sunkit_image.coalignment import mapsequence_coalign_by_rotation
https://docs.sunpy.org/en/stable/whatsnew/4.0.html
'''
import stackplotX as stp

Download full disk AIA maps (update_pointing calibration from aia_prep can be only applied to full-disk images)

In [2]:
# Check if directory is empty
data_path = '/home/ivan/sunpy/data/'

In [3]:
#
start_time = Time('2011-03-07T14:13:11', scale='utc', format='isot')
jsoc_email = "ido4@njit.edu"  # Credentials for JSOC database
# 2011-03-07T14:13:11
# Getting the list of directories

data_path = '/home/ivan/sunpy/data/'
if len(os.listdir(data_path)) == 0:
    query = Fido.search(
        a.Time(start_time - 0*u.h, start_time + 1*u.hour),
        a.Wavelength(131*u.angstrom),
        a.Sample(12*u.s),
        a.jsoc.Series.aia_lev1_euv_12s,
        a.jsoc.Notify(jsoc_email),
        #a.jsoc.Segment.image,
        #cutout,
    )

    print(query)
    files = Fido.fetch(query)
    files.sort()

Results from 1 Provider:

301 Results from the JSOCClient:
Source: http://jsoc.stanford.edu

       T_REC         TELESCOP INSTRUME WAVELNTH CAR_ROT
-------------------- -------- -------- -------- -------
2011-03-07T14:13:02Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:13:14Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:13:26Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:13:38Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:13:50Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:14:02Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:14:14Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:14:26Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:14:38Z  SDO/AIA    AIA_1      131    2107
2011-03-07T14:14:50Z  SDO/AIA    AIA_1      131    2107
                 ...      ...      ...      ...     ...
2011-03-07T15:11:02Z  SDO/AIA    AIA_1      131    2107
2011-03-07T15:11:14Z  SDO/AIA    AIA_1      131    2107
2011-03-07T15:11:26Z  SDO/AIA    AIA_1      131    2107
2011-03-07T

Files Downloaded:   0%|          | 0/602 [00:00<?, ?file/s]

aia.lev1_euv_12s.2011-03-07T141311Z.131.image_lev1.fits:   0%|          | 0.00/8.86M [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-03-07T141311Z.131.spikes.fits:   0%|          | 0.00/69.1k [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-03-07T141325Z.131.image_lev1.fits:   0%|          | 0.00/8.11M [00:00<?, ?B/s]

...

aia.lev1_euv_12s.2011-03-07T151247Z.131.spikes.fits:   0%|          | 0.00/63.4k [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-03-07T151259Z.131.image_lev1.fits:   0%|          | 0.00/8.80M [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-03-07T151259Z.131.spikes.fits:   0%|          | 0.00/63.4k [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-03-07T151311Z.131.image_lev1.fits:   0%|          | 0.00/8.80M [00:00<?, ?B/s]

aia.lev1_euv_12s.2011-03-07T151311Z.131.spikes.fits:   0%|          | 0.00/63.4k [00:00<?, ?B/s]

Loading AIA maps

In [3]:
obs_data_path = '/media/ivan/TOSHIBA EXT/aia_img/maps_part/'
obs_full_disk = '/media/ivan/TOSHIBA EXT/aia_img/2011_event/full_disk/'
obs_lvl15_data = '/media/ivan/TOSHIBA EXT/aia_img/2011_event/full_disk/calibrated/'
start_time = Time('2011-03-07T13:45:26', scale='utc', format='isot')

In [12]:
print(obs_full_disk)

/media/ivan/TOSHIBA EXT/aia_img/2011_event/full_disk/


In [8]:
mapsequence.exposure_time

AttributeError: 'MapSequence' object has no attribute 'exposure_time'

Run pointing and rotation calibrations to upgrade to level 1.5

In [4]:
from aiapy.calibrate import normalize_exposure, register, update_pointing

#mapsequence = sunpy.map.Map(obs_data_path + '/aia.lev1_euv_12s.2011-03-07T*.fits', sequence=True)
# Full-disk images
mapsequence = sunpy.map.Map(obs_full_disk + '/aia.lev1_euv_12s.2011-03-07T*.fits', sequence=True)

lvl_15_maps = []
for map in mapsequence.maps:
    print(type(map))
    m_updated_pointing = update_pointing(map)
    m_registered = register(m_updated_pointing)
    cal_map = normalize_exposure(m_registered)
    lvl_15_maps.append(cal_map)

cal_map_sequence = sunpy.map.Map(lvl_15_maps, sequence=True)

# Write calibrated lvl 1.5 files to the FITS file output
for map in cal_map_sequence.maps:
    print(map.date)
    map.save('aia.lev1.5_euv_12s.'+str(map.date)+'.fits')

<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
<class 'sunpy.map.sources.sdo.AIAMap'>
2011-03-07T14:10:57.618
2011-03-07T14:11:09.619
2011-03-07T14:11:24.191
2011-03-07T14:11:33.617
2011-03-07T14:11:47.911
2011-03-07T14:11:57.618
2011-03-07T14:12:11.582
2011-03-07T14:12:21.618
2011-03-07T14:12:33.617
2011-03-07T14:12:45.625
2011-03-07T14:13:00.191
2011-03-07T14:13:09.618


In [ ]:
# Write calibrated lvl 1.5 files to the FITS file output
for map in cal_map_sequence.maps:
    print(map.date)
    map.save('aia.lev1.5_euv_12s.'+str(map.date)+'.fits')

In [9]:
import gc
# del cal_map_sequence
# del lvl_15_maps
del mapsequence, lvl_15_maps
gc.collect()

3121

In [5]:
# load all calibrated images and write maps with ROI that includes flare itself
cal_map_sequence = sunpy.map.Map(obs_lvl15_data + '/aia.lev1.5_euv_12s.2011-03-07T*.fits', sequence=True)
obs_lvl15_flare_data = '/media/ivan/TOSHIBA EXT/aia_img/2011_event/flare_roi/calibrated/'

In [6]:
print(obs_lvl15_data)

/media/ivan/TOSHIBA EXT/aia_img/2011_event/full_disk/calibrated/


In [7]:
for map in cal_map_sequence.maps:

    roi_bottom_left = SkyCoord(Tx=-500*u.arcsec, Ty=200*u.arcsec, frame=map.coordinate_frame)
    roi_top_right = SkyCoord(Tx=-150*u.arcsec, Ty=550*u.arcsec, frame=map.coordinate_frame)
    cusp_submap = map.submap(bottom_left=roi_bottom_left, top_right=roi_top_right)

    cusp_submap.save(obs_lvl15_flare_data+'aia.lev1.5_euv_12s_roi.'+str(map.date).replace(':', '')+'.fits')

In [8]:
# Create and calibrate sunpy mapsequence using make_mapseq from stackplotX:
#mapseq_lvl15 = stp.make_mapseq(trange=)

# Load calibrated map sequence cropped to the region containing flare itself
mapsequence = sunpy.map.Map(obs_lvl15_flare_data + 'aia.lev1.5_euv_12s_roi.2011-03-07T*.fits', sequence=True)

In [9]:
st = stp.Stackplot(mapsequence)  # Create a stackplotX Stackplot object

Use the mapseq_mkdiff method to create a difference map from your sequence.
'mode' refers to the type of difference you wish to calculate:
'rratio' for relative ratio,
'rdiff' for relative difference,
'dtrend' for detrend.
'dt' represents the time difference (in seconds) between the current frame and the reference frame.

In [10]:
st.mapseq_mkdiff(mode='rratio', dt=24.)

making the diff mapseq.....


100%|██████████| 26/26 [00:00<00:00, 44.25it/s]


ValueError: fp and xp are not of the same length.

MapSequence of 26 elements, with maps from AIAMap

Use the plot_mapseq method to visualize the difference map sequence.
Set 'diff=True' to indicate that a difference map is to be plotted.
'norm' refers to the color normalization option for the map.
Note: you can interact with the pop-up window to modify the map. A left-click adds a new anchor point for the slit on the map, while a right-click removes the latest modification.

In [16]:
st.plot_mapseq(diff=True, norm=colors.Normalize(vmin=0.1, vmax=2.4))
# Use diff=False to plot original map sequence from fits files
# keep log norm:
# norm=colors.LogNorm(vmin=1e1, vmax=8e2))
# norm=colors.Normalize(vmin=0.1, vmax=2.4)

<IPython.core.display.Javascript object>

[-500.09902006 -150.2999734   200.09934909  549.89855121]


<AxesSubplot: title={'center': ' SDO AIA 131.0 Angstrom 2011-03-07T15:18:09.632'}, xlabel='X-position [arcsec]', ylabel='Y-position [arcsec]'>

Use the plot_stackplot method to generate a stack plot from your difference map sequence. 'norm' refers to the color normalization option (should be the same as before), 'cmap' refers to your choice of color map, and 'uni_cm' ensures a uniform color map across the stack plot and the map.

In [20]:
# Examine cutslitbd object
# Write slit data to pickle file
# print(st.cutslit)
# with open('cutslit.pickle', 'wb') as f:
    # pickle.dump(st.cutslit, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('cutslit.pickle', 'rb') as f:
    # st.cutslit = pickle.load(f)
    setattr(st, 'cutslit', pickle.load(f))

print(st.cutslit['x'], st.cutslit['y'])

AttributeError: can't set attribute

In [21]:
st.cutslitbd.update()
st.plot_stackplot(norm=colors.Normalize(vmin=0.1, vmax=2.4), cmap='Greys', uni_cm=True)
# norm=colors.Normalize(vmin=0.1, vmax=2.4)

making the stack plot...


  0%|          | 0/26 [00:00<?, ?it/s]

The slice should have at least two anchoring points! Return -1


TypeError: 'int' object is not subscriptable

Serialize the stackplot dictionary

In [ ]:
st.stackplt_tofile('/media/ivan/TOSHIBA EXT/buffer/stackplot.p')

In [ ]:
file = open('/media/ivan/TOSHIBA EXT/buffer/stackplot.p', 'rb')
stkplot = pickle.load(file)
print(stkplot['cutslit'])